<div style="direction:rtl"> برای هر دو سوال علاوه بر فایل ipynb فایل py. هم برای اطمینان قرار داده شده است! </div>

In [50]:
import cv2
import numpy as np
import random as rnd


<div style="direction:rtl"> در این تابع اولین بلاک به طور کاملا تصادفی به عکس اضافه میشود. </div>

In [51]:
def add_first_block(texture, result):
  h = rnd.randrange(height - bs)
  w = rnd.randrange(width - bs) 
  block = texture[h:h + bs, w:w + bs]
  result[0:bs, 0:bs] = block
  return result

<div style="direction:rtl"> template matching با استفاده از روش normalized cross correlation انجام میشود.
بعد یکی از مچ هایی که به اندازه سایز بلاک است و شباهتی بیشتر از 0.8 دارد به عنوان جواب برگردانده میشود.</div>

In [ ]:
def template_matching(texture, cs):
  res_matrix = cv2.matchTemplate(texture, cs, cv2.TM_CCORR_NORMED)
  a = np.argwhere(res_matrix >= 0.8)
  r = rnd.randrange(len(a))
  top_left = a[r]
  h = top_left[0]
  w = top_left[1]
  while h + bs >= height or w + bs >= width:
    r = rnd.randrange(len(a))
    top_left = a[r]
    h = top_left[0]
    w = top_left[1]
  return texture[h:h + bs, w:w + bs]

<div style="direction:rtl"> مینیمم سه عدد و موقعیت آن نسبت به عدد وسط را برمیگرداند. </div>

In [ ]:
def minimum(a, b, c):
    if a < b and a <= c:
        return a, -1
    if b <= a and b <= c:
        return b, 0
    if c < a and c < b:
        return c, 1

<div style="direction:rtl"> در این تابع مین کات عمودی انجام میشود. ابتدا عکس سمت چپ و راست از هم کم میشوند 
 و به توان 2 می رسند. سپس مقادیر روی axis سوم جمع میشوند (درواقع مقدار آبی، قرمز و سبز برای هر پیکسل با هم جمع میشود). بعد با روش
 توضیح داده شده در کلاس روی آرایه محاسبه شده (ssd در کد پایین) مین کات اجرا میشود، و دو آرایه e و path برگردانده میشوند.
 برای مثال path[i][j] نشان می دهد که از چه خانه ای از سطر قبلی به این خانه بیاییم تا مقدار محاسبه شده در e کمینه باشد.</div>

In [ ]:
def min_cut_vertical(left, right):
  temp = (left - right) ** 2
  ssd = np.sum(temp, axis=2)
  e = np.zeros((bs, css), dtype=np.float32)
  path = np.zeros((bs, css))
  e[0] = ssd[0]
  for k in range(1, bs):
      for l in range(0, css):
          if l == 0:
              min_last = minimum(np.inf, e[k-1][l], e[k-1][l+1])
          elif l == css - 1:
              min_last = minimum(e[k-1][l-1], e[k-1][l], np.inf)
          else:
              min_last = minimum(e[k-1][l-1], e[k-1][l], e[k-1][l+1])
          e[k][l] = min_last[0] + ssd[k][l]
          path[k][l] = min_last[1]
  return (e, path)

<div style="direction:rtl"> این تابع با استفاده از نتیجه تابع بالا مسیر بهینه را پیدا میکند. ابتدا مقدار کمینه سطر آخر آرایه e را پیدا کرده
 سپس با استفاده از آرایه path مسیر را محاسبه میکند. در آخر لازم است که لیست برعکس شود چرا که مسیر از آخر محاسبه شده بود.</div>

In [ ]:
def find_min_path(cut_result):
    e = cut_result[0]
    path = cut_result[1]
    arg = np.argmin(e[bs-1])
    results = [arg]
    # for k in range(1, bs):
    #     results.append(int(path[bs - k - 1][results[k-1]]) + int(results[k-1]))
    results.extend(int(path[bs - k - 1][results[k-1]]) + int(results[k-1]) for k in range(1, bs))
    results.reverse()
    return results

<div style="direction:rtl"> این تابع فقط برای سطر اول استفاده و بلاک اول سطرهای دیگر استفاده میشود.
 کاری که این تابع انجام میدهد، مقداردهی به قسمت مشترک دو بلاک با استفاده از مسیر مشخص شده توسط دو تابع قبل است. برای مثال
 یه مسیر عمودی را در نظر بگیرید که در path مشخص شده، برای هر سطر این قسمت مشترک پیکسل هایی سمت چپ پیکسل موجود در مسیر برای آن سطر قرار دارند مقدارشان را از
 بلاک سمت چپ و در غیر اینصورت مقدارشان را از بلاک سمت راست میگیرند.</div>

In [ ]:
def compute_common_section(block, c_block, path, format):
    if format == 'vertical':
      section = np.zeros((bs, css, 3), dtype=np.uint8)
      for k in range(bs):
        temp = path[k]
        section[k, 0:temp + 1] = block[k, 0:temp + 1]
        section[k, temp + 1:css] = c_block[k, temp+1:css]
          # for l in range(css):
          #     if l <= path[k]:
          #         section[k][l] = block[k][l]
          #     else:
          #         section[k][l] = c_block[k][l]
      return section
    else:
      section = np.zeros((css, bs, 3), dtype=np.uint8)
      for k in range(bs):
        temp = path[k]
        section[0:temp + 1, k] = block[0:temp+1, k]
        section[temp+1:css, k] = c_block[temp+1:css, k]
          # for l in range(css):
              # if l <= path[k]:
              #     section[l][k] = block[l][k]
              # else:
              #     section[l][k] = c_block[l][k]
      return section
    

<div style="direction:rtl">  سطر اول را با استفاده از تابع های توضیح داده شده به عکس اضافه میکند. </div>

In [ ]:
def add_first_row(texture, result):
  result = add_first_block(texture, result)
  for k in range(1, j_range):
    block = np.zeros((bs,bs,3), dtype=np.uint8)
    cs = result[0:bs, k * (bs - css):k * (bs - css) + css] # common section
    new_block = template_matching(texture, cs)
    # cv2.imwrite('ap.jpg', new_block)
    cut_result = min_cut_vertical(cs, new_block[0:bs, 0:css])
    min_path = find_min_path(cut_result)
    new_common_section = compute_common_section(cs, new_block[0:bs, 0:css], min_path, 'vertical')
    block[:, 0:css] = new_common_section
    block[:, css:bs] = new_block[:, css:bs]
    result[0:bs, (bs - css) * k:(bs - css) * k + bs] = block
  return result

<div style="direction:rtl"> مین کات افقی را انجام میدهد. در واقع هر کدام از قسمت ها را 3 بار در جهت خلاف عقربه های ساعت 90 درجه
 میچرخاند و این دو قسمت جدید را به همان تابع min_cut_vertical میدهد تا مین کات را محاسبه کند. </div>

In [52]:
def min_cut_horizontal(up, down):
  up = np.rot90(up, 3)
  down = np.rot90(down, 3)
  return min_cut_vertical(up , down)

<div style="direction:rtl"> این تابع اولین بلاک ستونهای غیر اول را به عکس اضافه میکند. ابتدا توسط template matching قسمت های
 مشابه قسمت پایین بلاک بالایی پیدا میشوند، یکی از آنها به طور رندوم برگردانده میشود. مین کات بین قسمت مشترک بلاک قدیمی و جدید پیدا میشود. با استفاده از مین کات یافته شده
 قسمت مشترک محاسبه شده و بلاک جدید به عکس اضافه میشود.</div>

In [ ]:
def add_first_block_mid_row(texture, result, i):
  block = np.zeros((bs,bs,3), dtype=np.uint8)
  cs = result[i * (bs - css):i * (bs - css) + css, 0:bs] # cs = common section, css = common section size
  # cv2.imwrite('cs.jpg', cs)
  new_block = template_matching(texture, cs)
  cut_result = min_cut_horizontal(cs, new_block[0:css])
  min_path = find_min_path(cut_result)
  new_common_section = compute_common_section(cs, new_block[0:css], min_path, 'horizontal')
  block[0:css] = new_common_section
  block[css:bs] = new_block[css:bs]
  result[i * (bs - css) : i * (bs - css) + bs, 0:bs] = block
  return result

<div style="direction:rtl">  این تابع template matching ال (L) شکل را انجام میدهد.
 این عمل با استفاده از پارامتر mask تابع matchTemplate انجام میشود. maskی که پاس داده میشود، باید روی پیکسلهایی که میخواهیم روی آن template matching انجام شود مقدار غیر 0 داشته باشند. بقیه تابع مشابه تابع template_matching است.</div>

In [ ]:
def L_shaped_template_matching(texture, cs):
  mask = np.zeros((bs, bs, 3))
  mask[0:css] = [1, 1, 1]
  mask[:, 0:css] = [1, 1, 1]
  res_matrix = cv2.matchTemplate(texture, cs, cv2.TM_CCORR_NORMED, mask=mask)
  # min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res_matrix)
  a = np.argwhere(res_matrix >= 0.8)
  r = rnd.randrange(len(a))
  top_left = a[r]
  h = top_left[0]
  w = top_left[1]
  while h + bs >= height or w + bs >= width:
    # print('here')
    r = rnd.randrange(len(a))
    top_left = a[r]
    h = top_left[0]
    w = top_left[1]
  return texture[h:h + bs, w:w + bs]

<div style="direction:rtl"> در قسمت های Lشکل که یک مسیر افقی مین کات و یک مسیر عمودی مین کات داریم
این تابع نقطه برخورد این دو مسیر را در مربع بالا سمت چپ این قسمت Lشکل محاسبه میکند. برای محاسبه از این نکته استفاده میشود که مسیر عمودی همیشه در بالا و سمت راست مسیر افقی قرار دارد و هر جا که
مسیر عمودی به پایین یا سمت چپ مسیر افقی آمد، آن نقطه به عنوان نقطه برخورد برگردانده میشود.</div>

In [ ]:
def find_intersection_point(vertical_path, horizontal_path):
  # intersection_point = [10, 10]
  x = np.arange(0, bs)
  xx = x - horizontal_path # should be positive
  yy = vertical_path - x # should be negative
  xx_zero = np.where(xx==0)
  yy_zero = np.where(yy==0)
  if len(np.intersect1d(xx_zero[0], yy_zero[0])) != 0:
    return xx_zero[0][0], yy_zero[0][0]
  xx_positive = np.where(xx > 0)
  yy_negative = np.where(yy < 0)
  if xx_positive[0][0] < yy_negative[0][0]:
    return xx_positive[0][0], vertical_path[xx_positive[0][0]]
  else:
    return horizontal_path[yy_negative[0][0]], yy_negative[0][0]
  

<div style="direction:rtl"> از این تابع برای بلاک های سطرها و ستونهای غیر اول استفاده میشود. در ابتدا نقطه برخورد دو
 مسیر عمودی و افقی در قسمت Lشکل محاسبه میشود. سپس پیکسلهای بالا و سمت چپ این نقطه، مقدارشان را از خود عکس میگیرند. اما در حالتهای دیگر با توجه به اینکه پیکسل بالا یا پایین مسیر افقی و یا چپ یا راست مسیر عمودی قرار دارد، مقداردهی انجام میشود.</div>

In [ ]:
def compute_new_block(old_block, new_block, vertical_path, horizontal_path):
  intersection_point = find_intersection_point(vertical_path, horizontal_path)
  x = intersection_point[0]
  y = intersection_point[1]
  # print(intersection_point)
  # print(x)
  # print(y)
  block = np.zeros((bs, bs, 3), dtype=np.uint8)
  for k in range(css):
    for l in range(css):
      if k <= x and l <= y:
        block[k][l] = old_block[k][l]
      if k > x:
        if l <= vertical_path[k]:
          block[k][l] = old_block[k][l]
        else:
          block[k][l] = new_block[k][l]
      elif l > y:
        if k <= horizontal_path[l]:
          block[k][l] = old_block[k][l]
        else:
          block[k][l] = new_block[k][l]
  
  for k in range(css, bs):
    # temp = vertical_path[k]
    # block[k, 0:temp+1] = old_block[k, 0:temp+1]
    # block[k, temp+1:css] = new_block[k ,temp+1:css]
    for l in range(css):
        if l <= vertical_path[k]:
            block[k][l] = old_block[k][l]
        else:
            block[k][l] = new_block[k][l]
    
  for k in range(css, bs):
    # temp = horizontal_path[k]
    # block[0:temp+1, k] = old_block[0:temp+1, k]
    # block[temp+1:css, k] = new_block[temp+1:css, k]
      for l in range(css):
          if l <= horizontal_path[k]:
              block[l][k] = old_block[l][k]
          else:
              block[l][k] = new_block[l][k]
  block[css:bs, css:bs] = new_block[css:bs, css:bs]
  return block
  

<div style="direction:rtl">  سطرهای غیر اول را به عکس اضافه میکند. ابتدا قسمت مشترک را از عکس برمیدارد و با استفاده از 
 template matching بلاکهای مشابه را پیدا میکند. بعد با استفاده از بلاک جدید مین کات قسمت Lشکل
  (یک مسیر افقی و یک مسیر عمودی) را می یابد. بعد با استفاده از این مسیر و نقطه برخورد دو مسیر مقداردهی های لازم انجام میشود و بلاک جدید به عکس اضافه میشود.</div>

In [ ]:
def add_mid_row(texture, result, i):
  result = add_first_block_mid_row(texture, result, i)
  for k in range(1, j_range):
    # print(k)
    cs = result[i * (bs - css): i * (bs - css) + bs, k * (bs - css): k * (bs - css) + bs]
    new_block = L_shaped_template_matching(texture, cs)
    vertical_cs = result[i * (bs - css): i * (bs - css) + bs, k * (bs - css):k * (bs - css) + css]
    horizontal_cs = result[i * (bs - css): i * (bs - css) + css, k * (bs - css):k * (bs - css) + bs]
    vertical_cut_result = min_cut_vertical(vertical_cs, new_block[:, 0:css])
    horizontal_cut_result = min_cut_horizontal(horizontal_cs, new_block[0:css])
    vertical_min_path = find_min_path(vertical_cut_result)
    horizontal_min_path = find_min_path(horizontal_cut_result)
    block = compute_new_block(cs, new_block, vertical_min_path, horizontal_min_path)
    result[i * (bs - css):i * (bs - css) + bs, k * (bs - css): k * (bs - css) + bs] = block
  return result
    

<div style="direction:rtl"> اندازه بلاک 200 و اندازه قسمت مشترک 30 در نظر گرفته شده است. نتیجه در res01.jpg ذخیره میشود.
 البته بدیهی است به علت تصادفی بودن فرآیند، هر بار یک نتیجه جدید به دست خواهد آمد. لازم به ذکر است که یک نتیجه دیگر با اندازه بلاک 100 و اندازه قسمت مشترک 15 هم با نام res01-1.jpg در فایل ها موجود است. البته مقدار آستانه موجود در دو تابع template matching هم به ترتیب به 0.95 و 0.8 تغییر کرد. علت انتخاب بلاک سایز 200 هم به دلیل سرعت اجرا بود، چون با اندازه 100 تقریبا 5 دقیقه ران شدن کد طول میکشید که در این صورت زمان اجرا تقریبا 1 دقیقه است.</div>

In [ ]:

texture = cv2.imread('./texture1.jpg')
result = np.zeros((3000, 3000, 3), dtype=np.uint8)
bs = 200 # block size
css = 30 # common section size

height = texture.shape[0]
width = texture.shape[1]
i_range = ((2600 - bs) // (bs - css)) + 1
j_range = ((2600 - bs) // (bs - css)) + 1
# print(j_range)
# import time
# start_time = time.time()

for i in range(i_range):
  # print(i)
  if i == 0:
    result = add_first_row(texture, result)
  else:
    result = add_mid_row(texture, result, i)
# print("--- %s seconds ---" % (time.time() - start_time))
cv2.imwrite('res01.jpg', result[0:2500, 0:2500])
